# Import tools

In [1]:
!pip install gingerit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import spacy
from gingerit.gingerit import GingerIt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Reading the text files

In [7]:
txt=[]
from google.colab import files
uploaded_files = files.upload()

for file in uploaded_files.keys():
  with open(file) as f:
    for line in f:
      line = line.strip()
      if line=='----------': break
      if line!='':txt.append(line)

print("number of lines = ", len(txt))



Saving Statistics.txt to Statistics.txt
number of lines =  549


# Cleaning the text

In [8]:
def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt+=words
    return cleaned_txt

cleaned_txt = clean_txt(txt)
print("number of words = ", len(cleaned_txt))

number of words =  2293


## Training the model

In [9]:
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_stories)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1

    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total

    return markov_model

In [10]:
markov_model = make_markov_model(cleaned_txt)
print("number of states = ", len(markov_model.keys()))
markov_model.keys()

number of states =  1600


dict_keys(['a level', 'level maths', 'maths statistics', 'statistics revision', 'revision notes', 'notes planning', 'planning and', 'and data', 'data collection', 'collection problem', 'problem specification', 'specification and', 'and analysis', 'analysis what', 'what is', 'is the', 'the purpose', 'purpose of', 'of the', 'the investigation', 'investigation what', 'what data', 'data is', 'is needed', 'needed how', 'how will', 'will the', 'the data', 'data be', 'be used', 'used data', 'collection how', 'be collected', 'collected how', 'will bias', 'bias be', 'be avoided', 'avoided what', 'what sample', 'sample size', 'size is', 'needed processing', 'processing and', 'and representing', 'representing how', 'be cleaned', 'cleaned which', 'which measures', 'measures will', 'will be', 'be calculated', 'calculated how', 'be represented', 'represented interpreting', 'interpreting and', 'and discussing', 'discussing data', 'collection types', 'types of', 'of data', 'data categorialqualitative'

# Generate Text

In [12]:
def generate_story(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story += curr_state + " "
    while n < limit:
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values()))

        curr_state = next_state[0]
        story += curr_state + " "
        n += 1
    # story = add_punctuation(story)
    return story

In [13]:
def grammaticize_text(text):
    ginger_parser = GingerIt()
    result = ginger_parser.parse(text)
    return result['result']

In [14]:
def grammaticize_text1(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    # sentences=grammaticize_text(sentences)
    return ".".join(sentences)


In [20]:
story=generate_story(markov_model, start="upper quartile", limit=30)
result=grammaticize_text1(story)
sent=grammaticize_text(result)
# gramm_story=sent.replace(".\n", ".")
print("Generated Story: ", sent)

Generated Story:  upper quartile lower quartile middle value of data less than x PX x calculate the value the more stringent the test state the distribution model used in the test checks whether 𝜌𝜌 is close to or n is large if the conditions are true then xbnp can be approximated by x without continuity correction pxPXfSFith continuity correction pxPXfSFwwmathsboxorguk sampling
